Notebook Bag
============



## Preliminaries



This notebook requires the installation of the package `mimesis`.
Install it with `pip` or `conda`, for the latter run
`conda install -c conda-forge mimesis`.



In [ ]:
import platform
import os

# Get the current operating system
os = platform.system()


## Start the Dask Client



Starting the Dask Client is optional. It will provide a dashboard which is useful to gain insight on the computation.

The link to the dashboard will become visible when you create the client below. We recommend having it open on one side of your screen while using your notebook on the other side. This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.



In [ ]:
from dask.distributed import Client, progress

client = Client(n_workers=4, threads_per_worker=1)
client


Address of the scheduler



In [ ]:
client.scheduler_info()


Notes:

-   the connection string (ip:port) of the scheduler can be used to connect to an existing cluster
-   a cluster can be shutdown with `client.shutdown()`



## Prepare Data



In [ ]:
import dask
import json
import os

os.makedirs("data/bag", exist_ok=True)  # Create data/ directory

b = dask.datasets.make_people(
    npartitions=10, records_per_partition=1000  # Make records of people,
)  # with default values
b.map(json.dumps).to_textfiles("data/bag/*.json")  # Encode as JSON, write to disk


Take a quick look



In [ ]:
if os == 'Windows':
    !dir /b data\bag\*.json
elif os == 'Linux':
    !ls -lah data/bag/*.json

## Load Data



First look at the raw data:



In [ ]:
if os == 'Windows':
    !Get-Content data\bag\0.json -TotalCount 2
elif os == 'Linux':
    !head -n 2 data/bag/0.json

Load it with dask:



In [ ]:
import dask.bag as db

b = db.read_text("data/bag/*.json").map(json.loads)
b


How many entries are there?



In [ ]:
b.count().compute()


Take two elements (from the first partition)



In [ ]:
b.take(2)


Extract some information from each entry:



In [ ]:
b.map(lambda record: record["occupation"]).take(2)


To get a list of all distinct occupations, use the function `distinct`.



In [ ]:
%%time
b.map(lambda record: record['occupation']).distinct().compute()

What is the difference to this approach?



In [ ]:
%%time
set(b.map(lambda record: record['occupation']).take(1000, npartitions=-1))

## Map, Filter, Aggregate



We can process this data by filtering out only certain records of interest, mapping functions over it to process our data, and aggregating those results to a total value.



In [ ]:
b.filter(lambda record: record["age"] > 30).take(2)  # Select only people over 30


## Chain Computations



It is common to do many of these steps in one pipeline, only calling compute or take at the end.



In [ ]:
result = (
    b.filter(lambda record: record["age"] > 30)
    .map(lambda record: record["occupation"])
    .frequencies(sort=True)
    .topk(15, key=1)
)
result


As with all lazy Dask collections, we need to call `compute` to actually evaluate our result. The `take` method used in earlier examples is also like `compute` and will also trigger computation.



In [ ]:
result.compute()


## Convert to Dask DataFrames



Dask Bags are good for reading in initial data, doing a bit of pre-processing, and then handing off to some other more efficient form like Dask Dataframes. Dask Dataframes use Pandas internally, and so can be much faster on numeric data and also have more complex algorithms.

However, Dask Dataframes also expect data that is organized as flat columns. It does not support nested JSON data very well (Bag is better for this).

Here we make a function to flatten down our nested data structure, map that across our records, and then convert that to a Dask Dataframe.



In [ ]:
def flatten(record):
    return {
        "age": record["age"],
        "occupation": record["occupation"],
        "telephone": record["telephone"],
        "credit-card-number": record["credit-card"]["number"],
        "credit-card-expiration": record["credit-card"]["expiration-date"],
        "name": " ".join(record["name"]),
        "street-address": record["address"]["address"],
        "city": record["address"]["city"],
    }


b.map(flatten).take(1)


In [ ]:
df = b.map(flatten).to_dataframe()
df.head()


## Task



Count the number of people with expired credit cards.
Do not use the data frame form the previous task, instead use
the original bag `b` and apply a filter to it.
Finally provide a **pandas** DataFrame with columns
`name`, `street` and `city` that contains all these people
(Note: calling `compute` for a Dask DataFrame will return a Pandas DataFrame).

